In [2]:
###TRANFORMATION FOR SURVEY DATA
##TODO: Log messages for required Columns, date formating etc 
##TODO: Modify the React to receive the error log https://chatgpt.com/c/aafeb9ec-24b3-4a0f-876d-343c4f70a1cc

import pandas as pd
from textblob import TextBlob
from langdetect import detect
from langdetect.lang_detect_exception import LangDetectException
from datetime import datetime
import logging
import google.cloud.logging
from google.cloud.logging.handlers import CloudLoggingHandler


# Load the dataset
file_path = r'/Users/phonavitra/Desktop/term 5/Service Studio/raw part 1/Vickers Onboarding.csv'
data = pd.read_csv(file_path)

def is_english(text):
    try:
        return detect(text) == 'en'
    except LangDetectException:
        return False
    
##formating data for the survey data
def format_date(date_str):
    try:
        # Check if it's NaN or float first
        if pd.isna(date_str) or isinstance(date_str, float):
            return date_str  # Return NaN or float as-is
        
        # Convert to string and strip any whitespace
        date_str = str(date_str).strip()
        
        # Parse the date assuming it's in the format '22/4/2024 15:43'
        date_obj = datetime.strptime(date_str, '%d/%m/%Y %H:%M')
        
        # Format date_obj into '22/04/2024'
        formatted_date = date_obj.strftime('%d/%m/%Y')
        return formatted_date
    except ValueError:
        return date_str  # Return original value if parsing fails


date_column = None
for col in data.columns:
    if 'date' in col.lower() or data[col].apply(lambda x: isinstance(x, str) and '/' in x).all():
        date_column = col
        data[col] = data[col].apply(format_date)
        break


question_texts = data.iloc[0]

# Identify columns to drop (only Q{int} and no NPS)
columns_to_drop = []
for col in data.columns:
    if not (col.startswith('Q') and col[1:].isdigit()):
        if col != date_column:
            columns_to_drop.append(col)
    else:
        if "NPS" in col or "rating" in col.lower() or "scale" in col.lower():
            columns_to_drop.append(col)
        elif pd.to_numeric(data[col][1:], errors='coerce').notna().all():
            columns_to_drop.append(col)
        elif data[col][1:].str.strip().str.lower().isin(['yes', 'no']).all():
            columns_to_drop.append(col)

# Drop identified columns
data.drop(columns=columns_to_drop, inplace=True)

# Drop the first row containing the question texts
data = data[1:]

# Filter out rows containing the specific entry after the question row
data = data[~data.apply(lambda row: row.astype(str).str.contains('QID').any(), axis=1)]

# Function to check if feedback is valid
def is_valid_feedback(feedback):
    if pd.isna(feedback):
        return False
    words = feedback.split()
    return len(feedback) > 1 and len(words) > 2

# Drop short and NaN feedbacks
for col in data.columns:
    data[col] = data[col].apply(lambda x: x if is_valid_feedback(x) and is_english(x) else None)

# Append question text to each feedback entry in the respective columns
for col in data.columns:
    if col.startswith('Q'):
        question_text = question_texts[col]
        data[col] = data[col].apply(lambda x: f"Vickers: {question_text}: {x}" if pd.notna(x) else x)
        # data[col] = data[col].apply(lambda x: f"{question_text}: {x}" if pd.notna(x) else x)


# Drop columns with all NaN values after feedback processing
data.dropna(axis=1, how='all', inplace=True)

# Convert the dataset from wide to long format without specifying id_vars
long_format_data = data.melt(var_name='Question Code', value_name='Feedback')

# Filter out rows where Feedback is NaN
long_format_data = long_format_data.dropna(subset=['Feedback'])

# Map all feedback to 'Secured Loan' and 'Auto/Car Loan'
long_format_data['Product'] = 'Investments'
long_format_data['Subcategory'] = 'Vickers'
long_format_data['Feedback Category'] = ''  
long_format_data['Sentiment'] = None  # Placeholder for sentiment analysis results
long_format_data['Sentiment Score'] = None
long_format_data['Source'] = 'Product Survey'

# If a date column was found, add it to the long_format_data DataFrame
if date_column:
    long_format_data['Date'] = data[date_column].values

# Define the desired column order
desired_columns = ['Date', 'Feedback', 'Product', 'Subcategory', 'Feedback Category', 'Sentiment', 'Sentiment Score', 'Source']

# Reorder columns using reindex
long_format_data = long_format_data.reindex(columns=desired_columns)

# Save the transformed dataset to a CSV file
output_file_path = r'/Users/phonavitra/Desktop/term 5/Service Studio/Transformed New/Transformed_Vickers.csv'
long_format_data.to_csv(output_file_path, index=False)

print("Data transformation complete. File saved to:", output_file_path)


Data transformation complete. File saved to: /Users/phonavitra/Desktop/term 5/Service Studio/raw part 1/Voice_Data.csv


In [ ]:

###FORMAT DATE 

file_path = '/Users/phonavitra/Desktop/term 5/Service Studio/Archive/Transformed/categorisation/categorisation1_fake.csv'
df = pd.read_csv(file_path)

def format_date(date_str):
    try:
        # Check if it's NaN or float first
        if pd.isna(date_str) or isinstance(date_str, float):
            return date_str  # Return NaN or float as-is
        
        # Convert to string and strip any whitespace
        date_str = str(date_str).strip()
        
        date_obj = datetime.strptime(date_str, '%d/%m/%y')
        
        # Format date_obj into 'day/month/two-digit-year'
        formatted_date = date_obj.strftime('%d/%m/%y')
        return formatted_date
    except ValueError:

        try:
            date_obj = datetime.strptime(date_str, '%d/%m/%Y')
            return date_obj.strftime('%d/%m/%y')
        except ValueError:
            return date_str  # Return original value if both parsing attempts fail

# Apply the function to format the Date column
df['Date'] = df['Date'].apply(format_date)

# Delete all entries in the "Sentiment" column
df['Sentiment'] = ""

# Define the function to generate random ratings and corresponding sentiments
def generate_sentiment_and_score():
    score = round(np.random.uniform(0.0, 5.0), 1)
    if 0.0 <= score <= 1.0:
        sentiment = 'Frustrated'
    elif 1.1 <= score <= 2.5:
        sentiment = 'Unsatisfied'
    elif 2.6 <= score <= 3.5:
        sentiment = 'Neutral'
    elif 3.6 <= score <= 4.5:
        sentiment = 'Satisfied'
    elif 4.6 <= score <= 5.0:
        sentiment = 'Excited'
    return score, sentiment

# Apply the function to generate random ratings and corresponding sentiments
df['Sentiment Score'], df['Sentiment'] = zip(*df.apply(lambda row: generate_sentiment_and_score(), axis=1))

# display(df)

# Save the modified DataFrame back to a CSV file
output_file_path = '/Users/phonavitra/Desktop/term 5/Service Studio/Archive/Transformed/categorisation/categorisation1_fake_data_w_sentiment_score.csv'
df.to_csv(output_file_path, index=False)


In [7]:
##TRANSFORM FOR VOICE CALL
import pandas as pd
from datetime import datetime
from langdetect import detect
from langdetect.lang_detect_exception import LangDetectException

# Load the dataset
file_path = '/Users/phonavitra/Desktop/term 5/Service Studio/raw part 1/Mar to May mock SR data .csv'
data = pd.read_csv(file_path)

# Function to format date to 'dd/mm/yyyy'
def format_date(date_str):
    try:
        # Check if it's NaN or float first
        if pd.isna(date_str) or isinstance(date_str, float):
            return date_str  # Return NaN or float as-is
        
        # Convert to string and strip any whitespace
        date_str = str(date_str).strip()
        
        date_obj = datetime.strptime(date_str, '%d/%m/%y')
        
        # Format date_obj into 'day/month/two-digit-year'
        formatted_date = date_obj.strftime('%d/%m/%y')
        return formatted_date
    except ValueError:

        try:
            date_obj = datetime.strptime(date_str, '%d/%m/%Y')
            return date_obj.strftime('%d/%m/%y')
        except ValueError:
            return date_str  # Return original value if both parsing attempts fail

# Function to check if text is in English
def is_english(text):
    try:
        return detect(text) == 'en'
    except LangDetectException:
        return False

# Check for date columns and format one as 'Date'
date_column = None
for col in data.columns:
    try:
        if data[col].apply(lambda x: isinstance(x, str) and '/' in x).all():
            data[col] = data[col].apply(format_date)
            date_column = col
            break
    except Exception as e:
        continue

# Check for columns containing 'feedback' or 'comments' (case insensitive)
feedback_column = None
for col in data.columns:
    if 'feedback' in col.lower() or 'comments' in col.lower():
        feedback_column = col
        break

def is_valid_feedback(feedback):
    if pd.isna(feedback):
        return False
    words = feedback.split()
    return len(feedback) > 1 and len(words) > 2

# Ensure the feedback column exists and filter non-English and NaN entries
if feedback_column:
    data = data[data[feedback_column].apply(lambda x: pd.notna(x) and is_english(x) and is_valid_feedback(x))]

# Define additional columns
data['Product'] = None
data['Subcategory'] = None
data['Feedback Category'] = ''
data['Sentiment'] = None
data['Sentiment Score'] = None
data['Source'] = 'Voice Call'

# Ensure desired column order
desired_columns = ['Date', 'Feedback', 'Product', 'Subcategory', 'Feedback Category', 'Sentiment', 'Sentiment Score', 'Source']

# Rename the feedback column to 'Feedback'
if feedback_column:
    data.rename(columns={feedback_column: 'Feedback'}, inplace=True)

# Rename the date column to 'Date'
if date_column:
    data.rename(columns={date_column: 'Date'}, inplace=True)

# Reorder columns using reindex
data = data.reindex(columns=desired_columns)

# Save the transformed dataset to a CSV file
output_file_path = '/Users/phonavitra/Desktop/term 5/Service Studio/raw part 1/Voice_Data.csv'
data.to_csv(output_file_path, index=False)

print("Data transformation complete. File saved to:", output_file_path)


Data transformation complete. File saved to: /Users/phonavitra/Desktop/term 5/Service Studio/raw part 1/Voice_Data.csv


In [7]:
###TRANSFORMATION FOR 5-STAR
import pandas as pd
from langdetect import detect
from langdetect.lang_detect_exception import LangDetectException
from datetime import datetime

# Load the dataset
file_path = r'/Users/phonavitra/Desktop/term 5/Service Studio/raw part 1/5star mock data.csv'
data = pd.read_csv(file_path)

# Function to detect if text is in English
def is_english(text):
    try:
        return detect(str(text)) == 'en'
    except LangDetectException:
        return False

# Formatting date
def format_date(date_str):
    try:
        # Check if it's NaN or float first
        if pd.isna(date_str) or isinstance(date_str, float):
            return date_str  # Return NaN or float as-is
        
        # Convert to string and strip any whitespace
        date_str = str(date_str).strip()
        
        # Attempt to parse the date assuming a two-digit year
        try:
            date_obj = datetime.strptime(date_str, '%d/%m/%y %H:%M')
            return date_obj.strftime('%d/%m/%y')
        except ValueError:
            # If the above fails, try parsing with a four-digit year
            date_obj = datetime.strptime(date_str, '%d/%m/%Y %H:%M')
            return date_obj.strftime('%d/%m/%y')
        
    except ValueError:
        return date_str

# Find and format the date column
date_column = None
for col in data.columns:
    if 'date' in col.lower() or data[col].apply(lambda x: isinstance(x, str) and '/' in x).all():
        date_column = col
        data[col] = data[col].apply(format_date)
        break

# Identify columns to drop based on criteria
columns_to_drop = []
for col in data.columns:
    if "NPS" in col or "rating" in col.lower() or "scale" in col.lower():
        columns_to_drop.append(col)
    elif pd.to_numeric(data[col][1:], errors='coerce').notna().all():
        columns_to_drop.append(col)
    elif data[col][1:].str.strip().str.lower().isin(['yes', 'no']).all():
        columns_to_drop.append(col)

# Drop identified columns
data.drop(columns=columns_to_drop, inplace=True)


# Identify the feedback column
feedback_column = None
for col in data.columns:
    if 'feedback' in col.lower() or 'comments' in col.lower():
        feedback_column = col
        break

def is_valid_feedback(feedback):
    if pd.isna(feedback):
        return False
    words = feedback.split()
    return len(feedback) > 1 and len(words) > 2

# Ensure the feedback column exists and filter non-English and NaN entries
if feedback_column:
    data = data[data[feedback_column].apply(lambda x: pd.notna(x) and is_english(x) and is_valid_feedback(x))]

# Define additional columns
data['Product'] = None
data['Subcategory'] = None
data['Feedback Category'] = ''
data['Sentiment'] = None
data['Sentiment Score'] = None ###I CAN CONSIDER USING THE 5 STAR SCORE DIRECTLY
data['Source'] = '5 Star Review'

# Ensure desired column order
desired_columns = ['Date', 'Feedback', 'Product', 'Subcategory', 'Feedback Category', 'Sentiment', 'Sentiment Score', 'Source']

# Rename the feedback column to 'Feedback'
if feedback_column:
    data.rename(columns={feedback_column: 'Feedback'}, inplace=True)

# Rename the date column to 'Date'
if date_column:
    data.rename(columns={date_column: 'Date'}, inplace=True)

# Reorder columns using reindex
data = data.reindex(columns=desired_columns)

# Save the transformed dataset to a CSV file
output_file_path = '/Users/phonavitra/Desktop/term 5/Service Studio/raw part 1/5star_transformed mock data.csv'
data.to_csv(output_file_path, index=False)

print("Data transformation complete. File saved to:", output_file_path)



Data transformation complete. File saved to: /Users/phonavitra/Desktop/term 5/Service Studio/raw part 1/5star_transformed mock data.csv


In [36]:
###TRANSFORMATION FOR SOCIAL MEDIA
import pandas as pd
from langdetect import detect
from langdetect.lang_detect_exception import LangDetectException
from datetime import datetime

# Load the dataset
file_path = r'/Users/phonavitra/Desktop/term 5/Service Studio/raw part 1/Social Media mock data.csv'
data = pd.read_csv(file_path)

# Function to detect if text is in English
def is_english(text):
    try:
        return detect(str(text)) == 'en'
    except LangDetectException:
        return False

# Formatting date
def format_date(date_str):
    try:
        # Check if it's NaN or float first
        if pd.isna(date_str) or isinstance(date_str, float):
            return date_str  # Return NaN or float as-is
        
        # Convert to string and strip any whitespace
        date_str = str(date_str).strip()
        
        date_obj = datetime.strptime(date_str, '%d/%m/%y')
        
        # Format date_obj into 'day/month/two-digit-year'
        formatted_date = date_obj.strftime('%d/%m/%y')
        return formatted_date
    except ValueError:

        try:
            date_obj = datetime.strptime(date_str, '%d/%m/%Y')
            return date_obj.strftime('%d/%m/%y')
        except ValueError:
            return date_str  # Return original value if both parsing attempts fail
        

# Find and format the date column
date_column = None
for col in data.columns:
    if 'date' in col.lower() and data[col].apply(lambda x: isinstance(x, str) and '/' in x).all():
        data[col] = data[col].apply(format_date)
        date_column = col
        break


# Identify the feedback column
feedback_column = None
for cols in data.columns:
    if 'feedback' in cols.lower() or 'comments' in cols.lower() or 'mentions' in cols.lower():
        feedback_column = cols
        break

def is_valid_feedback(feedback):
    if pd.isna(feedback):
        return False
    words = feedback.split()
    return len(feedback) > 1 and len(words) > 2

# Ensure the feedback column exists and filter non-English and NaN entries
if feedback_column:
    data = data[data[feedback_column].apply(lambda x: pd.notna(x) and is_english(x) and is_valid_feedback(x))]

# Define additional columns
data['Product'] = None
data['Subcategory'] = None
data['Feedback Category'] = ''
data['Sentiment'] = None ###I CAN CONSIDER USING SENTIMENT DIRECTLY
data['Sentiment Score'] = None 
data['Source'] = 'Social Media'

# Ensure desired column order
desired_columns = ['Date', 'Feedback', 'Product', 'Subcategory', 'Feedback Category', 'Sentiment', 'Sentiment Score', 'Source']

# Rename the feedback column to 'Feedback'
if feedback_column:
    data.rename(columns={feedback_column: 'Feedback'}, inplace=True)

# Rename the date column to 'Date'
if date_column:
    data.rename(columns={date_column: 'Date'}, inplace=True)

# Reorder columns using reindex
data = data.reindex(columns=desired_columns)

# Save the transformed dataset to a CSV file
output_file_path = '/Users/phonavitra/Desktop/term 5/Service Studio/raw part 1/Social Media_TRANSFORMED.csv'
data.to_csv(output_file_path, index=False)

print("Data transformation complete. File saved to:", output_file_path)

0     Customer posted in X that his cancellation for...
1     Netizens have reported that they are unable to...
2     Customer posted in X claiming that she has ema...
3     Forum thread started by a netizen who claims t...
4     Forum discussion started by a netizen who rece...
                            ...                        
79    Netizen is claiming that remittance was not re...
80    Netizens shared on Reddit about not being able...
81    Customer aired his sentiments on one of our br...
82    Forum thread that started as crowdsourcing on ...
83    Blog article about DBS Vantage Card’s features...
Name: Mentions, Length: 84, dtype: object


,Date,Feedback,Product,Subcategory,Feedback Category,Sentiment,Sentiment Score,Source
0,01/03/24,Customer posted in X that his cancellation for...,None,None,,None,None,Social Media
1,01/03/24,Netizens have reported that they are unable to...,None,None,,None,None,Social Media
2,04/03/24,Customer posted in X claiming that she has ema...,None,None,,None,None,Social Media
3,07/03/24,Forum thread started by a netizen who claims t...,None,None,,None,None,Social Media
4,08/03/24,Forum discussion started by a netizen who rece...,None,None,,None,None,Social Media
...,...,...,...,...,...,...,...,...
79,20/05/24,Netizen is claiming that remittance was not re...,None,None,,None,None,Social Media
80,24/05/24,Netizens shared on Reddit about not being able...,None,None,,None,None,Social Media
81,24/05/24,Customer aired his sentiments on one of our br...,None,None,,None,None,Social Media
82,28/05/24,Forum thread that started as crowdsourcing on ...,None,None,,None,None,Social Media


Data transformation complete. File saved to: /Users/phonavitra/Desktop/term 5/Service Studio/raw part 1/Social Media_TRANSFORMED.csv


In [38]:
def parse_filename(filename):
    # Split the filename using the double underscore separator
    parts = filename.split('__')
    
    # Check if the filename is in the correct format
    if len(parts) < 3:
        raise ValueError("Filename format is incorrect. Expected format: 'product__source__fname'")
    
    # Extract the product and source values
    product = parts[0]
    source = parts[1]
    
    return product, source

# Example usage
filename = "product1__sourceA__datafile.txt"
product, source = parse_filename(filename)